In [2]:
!pip install pdf2image

DEPRECATION: seg-torch 0.1.7 has a non-standard dependency specifier tensorboardX>=2.0opencv-python. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of seg-torch or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
from pdf2image import convert_from_path, convert_from_bytes
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)
images = convert_from_path('test1.pdf')

In [6]:
for idx,i in enumerate(images):
    i.save(str(idx)+".jpg")
    

In [9]:
def convert_xyxyxyxy_2_xyxy(points):
    x1,y1=points[0]
    x2,y2=points[2]
    return x1,y1,x2,y2

In [17]:
from paddleocr import PaddleOCR,draw_ocr
import numpy as np
import cv2
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `french`, `german`, `korean`, `japan`
# to switch the language model in order.
ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and load model into memory
Results={"pos":[],"text":[]}
for idx,image in enumerate(images):
    image=np.array(image)
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    result = ocr.ocr(image, cls=True)
    for idx in range(len(result)):
        res = result[idx]
        for line in res:
            points=line[0]
            box=list(map(int,convert_xyxyxyxy_2_xyxy(points)))
            Results["pos"].append(box)
            Results["text"].append(line[1][0])
            print(line[1])


[2024/03/16 10:23:25] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\madri/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\madri/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None,

In [15]:
Results

{'pos': [[135, 63, 583, 97],
  [618, 63, 1551, 97],
  [758, 105, 1418, 136],
  [192, 117, 522, 151],
  [219, 153, 489, 193],
  [721, 144, 1457, 178],
  [108, 224, 372, 256],
  [1314, 231, 1511, 263],
  [108, 263, 320, 295],
  [106, 300, 1543, 336],
  [105, 342, 252, 370],
  [108, 378, 281, 412],
  [394, 375, 1568, 409],
  [106, 417, 273, 451],
  [138, 453, 268, 487],
  [505, 448, 733, 495],
  [874, 451, 1024, 485],
  [1245, 448, 1384, 493],
  [108, 492, 298, 526],
  [404, 495, 1181, 529],
  [140, 534, 411, 565],
  [504, 531, 768, 563],
  [874, 529, 1078, 565],
  [1244, 528, 1439, 568],
  [108, 570, 1122, 604],
  [1234, 562, 1309, 608],
  [139, 601, 317, 641],
  [505, 604, 711, 641],
  [877, 604, 1053, 641],
  [1243, 607, 1464, 641],
  [108, 648, 1462, 680],
  [212, 682, 586, 719],
  [138, 724, 364, 755],
  [505, 719, 758, 755],
  [876, 719, 1154, 755],
  [1243, 719, 1496, 758],
  [106, 758, 911, 794],
  [1014, 760, 1472, 792],
  [138, 799, 263, 833],
  [506, 794, 636, 834],
  [875, 794

In [18]:
Title=["Mark the letter A, B, C, or D on your answer sheet to indicate the correct answer to each of thefollowing questions","Mark the letter A, B, C, or D on your answer sheet to indicate the sentence that best completes each of the following exchanges.","Mark  the  letter  A,  B,  C,  or  D  on  your  answer  sheet  to indicate  the  word  CLOSEST  in  meaning  to  the underlined word in each of the following questions.","Mark the letter A, B, C, or D on your answer sheet to indicate the word(s) OPPOSITE in meaning to the underlined word(s) in each of the following question","Read the following passage and mark the letter A, B, C, or D on your answer sheet to indicate the correct word or phrase that best fits each of the numbered blanks from 26 to 30"]

In [21]:
'''
cách xử lý bài này
pdf->image->ocr->cluster theo bbox-> phân loại
dấu hiệu nhận biết Title,content và answers
Title: Mark the letter A,B,C,or D hoặc là Read the floowing passage
'''
Flag=False
Count=0
Cluster_question={}
for text,box in zip(Results["text"],Results["pos"]):
    
    if "Mark the letter A, B, C, or D".lower() in text.lower() or "Read the following passage and mark the letter A, B, C" in text.lower():
        Count += 1
        
        Cluster_question[f'Cluster_{Count}'].update({"pos":box,"text":text})
 
            
            
            
            
            
    

KeyError: 'Cluster_1'